# Evaluación del rendimiento

Dividir el dataframe en dos particiones:

- El conjunto de entrenamiento (75%- 80%)
- El conjunto de test (20% - 25%)

La intesección entre test y train es vacía

 Buscamos los k-vecinos más cercanos de cada uno de los elementos de test dentro del conjunto de entrenamiento, comparamos los resultados (tenemos el real) y así tenemos una medida de la 'bondad' de esta predicción.
 
 





In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
valencia = pd.read_csv('https://raw.githubusercontent.com/afoone/caipc-mar-2023/master/data/airbnb.csv')



In [3]:
## limpiar los datos

# corregir el price (tiene $ y no es numérico)
valencia['price'] = valencia['price'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')

valencia = valencia[valencia['price']<700]

In [4]:
# barajar

np.random.seed(3837980)
valencia = valencia.iloc[
    np.random.permutation(len(valencia))
]

In [5]:
# tenemos que crear un dataframe de train (80%) y uno de test (20%)

corte = math.trunc(len(valencia)*0.8)

In [6]:
train = valencia.iloc[:corte].copy()

In [7]:
test = valencia.iloc[corte:].copy()

In [8]:
train.shape

(5194, 75)

In [9]:
test.shape

(1299, 75)

In [10]:
def predict_price(df, mi_capacidad, k = 5):
    _df = df.copy()
    _df['distancia'] = _df['accommodates'].apply(
        lambda x: np.abs(mi_capacidad - x)
    )
    return _df.sort_values('distancia').iloc[0:k]['price'].mean()

In [11]:
test['predicted_price'] = test['accommodates'].apply(
    lambda x: predict_price(valencia, x)
)

In [12]:
train.shape[0]*test.shape[0]

6747006

In [13]:
test[['predicted_price', 'price']]

,predicted_price,price
3530,78.6,125.0
234,78.6,63.0
3945,32.6,40.0
6462,78.6,142.0
5006,115.4,158.0
...,...,...
5882,32.6,70.0
2961,66.8,21.0
1550,115.4,86.0
2312,32.6,34.0


In [22]:
(test['predicted_price']-test['price']).mean() # este valor no es bueno, compensa los negativos con los positivos

-16.241707467282527

In [15]:
(np.abs((test['predicted_price']-test['price']))).mean()

39.49682678983834

In [27]:
valencia[(valencia['price'] > 1000)* (valencia['accommodates']==1)]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
5076,648255583817027558,https://www.airbnb.com/rooms/648255583817027558,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,NaN,NaN,NaN,NaN,f,93,93,0,0,NaN
5147,648809313549112407,https://www.airbnb.com/rooms/648809313549112407,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,NaN,NaN,NaN,NaN,f,93,93,0,0,NaN
5090,648348424299859963,https://www.airbnb.com/rooms/648348424299859963,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,NaN,NaN,NaN,NaN,f,93,93,0,0,NaN
5069,648247685615181497,https://www.airbnb.com/rooms/648247685615181497,20221221170335,2022-12-22,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,5.0,4.5,5.0,NaN,f,93,93,0,0,0.45
5134,648426745255991424,https://www.airbnb.com/rooms/648426745255991424,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,NaN,NaN,NaN,NaN,f,93,93,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5152,648812332449803467,https://www.airbnb.com/rooms/648812332449803467,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,5.0,4.0,5.0,NaN,f,93,93,0,0,0.20
5064,648243584412460160,https://www.airbnb.com/rooms/648243584412460160,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,NaN,NaN,NaN,NaN,f,93,93,0,0,NaN
5141,648430772591629131,https://www.airbnb.com/rooms/648430772591629131,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,129230780,...,3.0,4.0,4.0,NaN,f,93,93,0,0,0.21
5203,649003049999248376,https://www.airbnb.com/rooms/649003049999248376,20221221170335,2022-12-21,previous scrape,Nice Studio close to the Ciudad de las Artes y...,Our accommodation is based around an innovativ...,"Valencia’s famous attraction, The City of Arts...",https://a0.muscache.com/pictures/prohost-api/H...,258154594,...,NaN,NaN,NaN,NaN,f,68,68,0,0,NaN


### MAE - Medium absolute error

In [25]:
mae_accommodates = (np.abs((test['predicted_price']-test['price']))).mean() #mae = medium absolute error

In [26]:
valencia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6493 entries, 5738 to 2287
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6493 non-null   int64  
 1   listing_url                                   6493 non-null   object 
 2   scrape_id                                     6493 non-null   int64  
 3   last_scraped                                  6493 non-null   object 
 4   source                                        6493 non-null   object 
 5   name                                          6493 non-null   object 
 6   description                                   6341 non-null   object 
 7   neighborhood_overview                         3567 non-null   object 
 8   picture_url                                   6493 non-null   object 
 9   host_id                                       6493 non-null 

In [27]:
def predict_price_dorms(df, mi_capacidad, k = 5):
    _df = df.copy()
    _df['distancia'] = _df['bedrooms'].apply(
        lambda x: np.abs(mi_capacidad - x)
    )
    return _df.sort_values('distancia').iloc[0:k]['price'].mean()

In [28]:
test['predicted_price_dorms'] = test['bedrooms'].apply(
    lambda x: predict_price_dorms(valencia, x)
)

In [31]:
mae_bedrooms = (
    np.abs(
        test['price'] - test['predicted_price_dorms']
    )
).mean()

In [32]:
mae_bedrooms


45.99919168591224

In [33]:
mae

39.49682678983834

In [34]:
test[['price', 'predicted_price', 'predicted_price_dorms']]

,price,predicted_price,predicted_price_dorms
3530,125.0,78.6,110.6
234,63.0,78.6,50.4
3945,40.0,32.6,50.4
6462,142.0,78.6,110.6
5006,158.0,115.4,174.4
...,...,...,...
5882,70.0,32.6,81.4
2961,21.0,66.8,50.4
1550,86.0,115.4,174.4
2312,34.0,32.6,50.4


### MSE Medium square root - Error al cuadrado medio

In [35]:
mse_accommodates = (
    (
        test['predicted_price']-test['price']
    )**2
).mean()

In [36]:
mse_accommodates

3764.898092443418

# RMSE Root of MSE

Acercarse al mae, desde un punto de vista 'humano', una estimación mas visible de 'cuantos euros suele separarse del valor real'




In [37]:
rmse_accommodates = mse_accommodates ** 0.5

In [38]:
rmse_accommodates 

61.35876540840288

In [39]:
rmse_bedrooms = (
    (
        test['predicted_price_dorms'] - test['price']
    )**2
).mean()**0.5

In [40]:
rmse_bedrooms


66.80637000243489

Diferencias entre MAE y RMSE

In [64]:
diferencias_uno = pd.Series(
    [5,10,5,10,5,10,5,10,5,10,5,10]
)

diferencias_dos = pd.Series(
    [0,0,0,0,0,0,0,0,0,75]
)

In [65]:
# mae 1
diferencias_uno.sum() / len(diferencias_uno)

7.5

In [66]:
# mae 2
diferencias_dos.sum() / len(diferencias_dos)

7.5

In [67]:
#rmse 1

((
    diferencias_uno**2
).sum()  / len(diferencias_uno) )**0.5

7.905694150420948

In [68]:
# rmse 2
((
    diferencias_dos**2
).sum()  / len(diferencias_dos) )**0.5

23.717082451262844